In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn import metrics
import torch.optim as optim

In [2]:
sub_features = ['42 tGravityAcc-mean()-Y',
 '43 tGravityAcc-mean()-Z',
 '51 tGravityAcc-max()-Y',
 '52 tGravityAcc-max()-Z',
 '54 tGravityAcc-min()-Y',
 '55 tGravityAcc-min()-Z',
 '56 tGravityAcc-sma()',
 '59 tGravityAcc-energy()-Z',
 '125 tBodyGyro-std()-Y',
 '128 tBodyGyro-mad()-Y',
 '138 tBodyGyro-energy()-Y',
 '165 tBodyGyroJerk-std()-Y',
 '168 tBodyGyroJerk-mad()-Y',
 '178 tBodyGyroJerk-energy()-Y',
 '181 tBodyGyroJerk-iqr()-Y',
 '425 fBodyGyro-mean()-Y',
 '428 fBodyGyro-std()-Y',
 '431 fBodyGyro-mad()-Y',
 '441 fBodyGyro-energy()-Y',
 '475 fBodyGyro-bandsEnergy()-1,8',
 '478 fBodyGyro-bandsEnergy()-25,32',
 '483 fBodyGyro-bandsEnergy()-1,16',
 '487 fBodyGyro-bandsEnergy()-1,24',
 '559 angle(X,gravityMean)',
 '560 angle(Y,gravityMean)',
 '561 angle(Z,gravityMean)']

act_features = ['4 tBodyAcc-std()-X',
 '7 tBodyAcc-mad()-X',
 '10 tBodyAcc-max()-X',
 '17 tBodyAcc-energy()-X',
 '202 tBodyAccMag-std()',
 '204 tBodyAccMag-max()',
 '215 tGravityAccMag-std()',
 '217 tGravityAccMag-max()',
 '266 fBodyAcc-mean()-X',
 '269 fBodyAcc-std()-X',
 '272 fBodyAcc-mad()-X',
 '275 fBodyAcc-max()-X',
 '282 fBodyAcc-energy()-X',
 '303 fBodyAcc-bandsEnergy()-1,8',
 '311 fBodyAcc-bandsEnergy()-1,16',
 '315 fBodyAcc-bandsEnergy()-1,24',
 '504 fBodyAccMag-std()',
 '505 fBodyAccMag-mad()',
 '506 fBodyAccMag-max()',
 '509 fBodyAccMag-energy()']

input_shape = len(sub_features) + len(act_features)

In [3]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 25),
            classifier_block(25, 20),
            classifier_block(20, 15),
            classifier_block(15, 10),
            nn.Linear(10, 3)
        )
    def forward(self, x):
        return self.network(x)

In [4]:
#defines each generator layer
#input and output dimensions needed
def generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace = True)
    )

#returns n_samples of z_dim (number of dimensions of latent space) noise
def get_noise(n_samples, z_dim):
    return torch.randn(n_samples, z_dim)

#defines generator class
class Generator(nn.Module):
    def __init__(self, z_dim = 10, feature_dim = input_shape, hidden_dim = 128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            generator_block(z_dim, 80),
            generator_block(80, 60),
            generator_block(60, 50),
            nn.Linear(50, feature_dim),
            nn.Tanh()
        )
    def forward(self, noise):
        return self.gen(noise)

def get_act_matrix(batch_size, a_dim):
    indexes = np.random.randint(a_dim, size = batch_size)
    
    one_hot = np.zeros((len(indexes), indexes.max()+1))
    one_hot[np.arange(len(indexes)),indexes] = 1
    return torch.Tensor(indexes).long(), torch.Tensor(one_hot)
    
def get_usr_matrix(batch_size, u_dim):
    indexes = np.random.randint(u_dim, size = batch_size)
    
    one_hot = np.zeros((indexes.size, indexes.max()+1))
    one_hot[np.arange(indexes.size),indexes] = 1
    return torch.Tensor(indexes).long(), torch.Tensor(one_hot)

def load_model(model, model_name):
    model.load_state_dict(torch.load(f'../../saved_models/{model_name}'))

In [5]:
#label is a list of integers specifying which labels to filter by
#users is a list of integers specifying which users to filter by
#y_label is a string, either "Activity" or "Subject" depending on what y output needs to be returned
def start_data(label, users, y_label, sub_features, act_features):
    #get the dataframe column names
    name_dataframe = pd.read_csv('../../data/features.txt', delimiter = '\n', header = None)
    names = name_dataframe.values.tolist()
    names = [k for row in names for k in row] #List of column names

    data = pd.read_csv('../../data/X_train.txt', delim_whitespace = True, header = None) #Read in dataframe
    data.columns = names #Setting column names
    
    X_train_1 = data[sub_features]
    X_train_2 = data[act_features]
    X_train = pd.concat([X_train_1, X_train_2], axis = 1)
    
    y_train_activity = pd.read_csv('../../data/y_train.txt', header = None)
    y_train_activity.columns = ['Activity']
    
    y_train_subject = pd.read_csv('../../data/subject_train.txt', header = None)
    y_train_subject.columns = ['Subject']
    
    GAN_data = pd.concat([X_train, y_train_activity, y_train_subject], axis = 1)
    GAN_data = GAN_data[GAN_data['Activity'].isin(label)]
    GAN_data = GAN_data[GAN_data['Subject'].isin(users)]
    
    X_train = GAN_data.iloc[:,:-2].values
    y_train = GAN_data[[y_label]].values
    
    return X_train, y_train.flatten()

# Activity Classifier

In [6]:
activities = [1, 3, 4]
users = [7, 8, 11]

X, y = start_data(activities, users, "Activity", sub_features, act_features)

In [7]:
for k in range(len(y)):
    if y[k] == 1:
        y[k] = 0
    elif y[k] == 3:
        y[k] = 1
    else:
        y[k] = 2

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)

model = Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [9]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 2.189556121826172, Final Batch Loss: 1.095049262046814
Epoch 2, Loss: 2.184676766395569, Final Batch Loss: 1.0925415754318237
Epoch 3, Loss: 2.18072509765625, Final Batch Loss: 1.0892794132232666
Epoch 4, Loss: 2.178658962249756, Final Batch Loss: 1.0903600454330444
Epoch 5, Loss: 2.1705089807510376, Final Batch Loss: 1.083122730255127
Epoch 6, Loss: 2.161190390586853, Final Batch Loss: 1.0746475458145142
Epoch 7, Loss: 2.1628228425979614, Final Batch Loss: 1.0839437246322632
Epoch 8, Loss: 2.15415620803833, Final Batch Loss: 1.0750361680984497
Epoch 9, Loss: 2.1447118520736694, Final Batch Loss: 1.0677019357681274
Epoch 10, Loss: 2.1410871744155884, Final Batch Loss: 1.0691503286361694
Epoch 11, Loss: 2.1366323232650757, Final Batch Loss: 1.0684109926223755
Epoch 12, Loss: 2.1290310621261597, Final Batch Loss: 1.0642881393432617
Epoch 13, Loss: 2.1199722290039062, Final Batch Loss: 1.0564165115356445
Epoch 14, Loss: 2.1068304777145386, Final Batch Loss: 1.05267834663391

Epoch 130, Loss: 0.19412896782159805, Final Batch Loss: 0.11375118046998978
Epoch 131, Loss: 0.18774664402008057, Final Batch Loss: 0.06718939542770386
Epoch 132, Loss: 0.15959352999925613, Final Batch Loss: 0.08644562214612961
Epoch 133, Loss: 0.21082370728254318, Final Batch Loss: 0.11432433873414993
Epoch 134, Loss: 0.1319495178759098, Final Batch Loss: 0.031981442123651505
Epoch 135, Loss: 0.11795098334550858, Final Batch Loss: 0.037660688161849976
Epoch 136, Loss: 0.1811750903725624, Final Batch Loss: 0.11313951015472412
Epoch 137, Loss: 0.1418714001774788, Final Batch Loss: 0.0686088502407074
Epoch 138, Loss: 0.12117725610733032, Final Batch Loss: 0.0513744130730629
Epoch 139, Loss: 0.16134177893400192, Final Batch Loss: 0.08252383768558502
Epoch 140, Loss: 0.12526001036167145, Final Batch Loss: 0.05009336769580841
Epoch 141, Loss: 0.13985393196344376, Final Batch Loss: 0.044275738298892975
Epoch 142, Loss: 0.2069656327366829, Final Batch Loss: 0.11274254322052002
Epoch 143, Loss

Epoch 257, Loss: 0.06775691825896502, Final Batch Loss: 0.052913010120391846
Epoch 258, Loss: 0.05590271111577749, Final Batch Loss: 0.004958777688443661
Epoch 259, Loss: 0.07293853908777237, Final Batch Loss: 0.016956612467765808
Epoch 260, Loss: 0.07440196350216866, Final Batch Loss: 0.024890299886465073
Epoch 261, Loss: 0.08742409572005272, Final Batch Loss: 0.0724361464381218
Epoch 262, Loss: 0.04399130027741194, Final Batch Loss: 0.010073269717395306
Epoch 263, Loss: 0.04938771761953831, Final Batch Loss: 0.011109927669167519
Epoch 264, Loss: 0.051086559891700745, Final Batch Loss: 0.021844036877155304
Epoch 265, Loss: 0.059496477246284485, Final Batch Loss: 0.03467296063899994
Epoch 266, Loss: 0.07428361847996712, Final Batch Loss: 0.04046666994690895
Epoch 267, Loss: 0.04884255863726139, Final Batch Loss: 0.011085854843258858
Epoch 268, Loss: 0.05101713538169861, Final Batch Loss: 0.01252610981464386
Epoch 269, Loss: 0.06534925568848848, Final Batch Loss: 0.011463952250778675
Ep

Epoch 381, Loss: 0.020436075516045094, Final Batch Loss: 0.004537581466138363
Epoch 382, Loss: 0.054730916395783424, Final Batch Loss: 0.03435564413666725
Epoch 383, Loss: 0.03402083180844784, Final Batch Loss: 0.008356388658285141
Epoch 384, Loss: 0.05636603198945522, Final Batch Loss: 0.033982083201408386
Epoch 385, Loss: 0.04131257254630327, Final Batch Loss: 0.01335382554680109
Epoch 386, Loss: 0.02340454515069723, Final Batch Loss: 0.00893782265484333
Epoch 387, Loss: 0.0630041528493166, Final Batch Loss: 0.048027537763118744
Epoch 388, Loss: 0.02884893026202917, Final Batch Loss: 0.0054559847339987755
Epoch 389, Loss: 0.016931725200265646, Final Batch Loss: 0.006495942827314138
Epoch 390, Loss: 0.02005172148346901, Final Batch Loss: 0.009427341632544994
Epoch 391, Loss: 0.035291386768221855, Final Batch Loss: 0.014745974913239479
Epoch 392, Loss: 0.05902056209743023, Final Batch Loss: 0.03455940634012222
Epoch 393, Loss: 0.021548299118876457, Final Batch Loss: 0.00566541217267513

Epoch 503, Loss: 0.014974287245422602, Final Batch Loss: 0.012766963802278042
Epoch 504, Loss: 0.010161842219531536, Final Batch Loss: 0.004617310129106045
Epoch 505, Loss: 0.014934339560568333, Final Batch Loss: 0.005777585320174694
Epoch 506, Loss: 0.017299523577094078, Final Batch Loss: 0.004304235801100731
Epoch 507, Loss: 0.013993288855999708, Final Batch Loss: 0.006233835592865944
Epoch 508, Loss: 0.03044567024335265, Final Batch Loss: 0.02368018962442875
Epoch 509, Loss: 0.008470481028780341, Final Batch Loss: 0.0013948052655905485
Epoch 510, Loss: 0.0208883136510849, Final Batch Loss: 0.0106586879119277
Epoch 511, Loss: 0.00462376547511667, Final Batch Loss: 0.0014730639522895217
Epoch 512, Loss: 0.006588720949366689, Final Batch Loss: 0.002061184262856841
Epoch 513, Loss: 0.008669006172567606, Final Batch Loss: 0.002223655115813017
Epoch 514, Loss: 0.019681390840560198, Final Batch Loss: 0.01491270586848259
Epoch 515, Loss: 0.01602329988963902, Final Batch Loss: 0.002699365140

...

Epoch 4513, Loss: 4.602713670465164e-05, Final Batch Loss: 1.7611930161365308e-05
Epoch 4514, Loss: 8.618882475275313e-06, Final Batch Loss: 5.994747880322393e-06
Epoch 4515, Loss: 0.00010216081864200532, Final Batch Loss: 4.406616426422261e-05
Epoch 4516, Loss: 6.0604877944570035e-05, Final Batch Loss: 2.2407533833757043e-05
Epoch 4517, Loss: 0.0013258337839943124, Final Batch Loss: 0.0013061584904789925
Epoch 4518, Loss: 0.00018780377922666958, Final Batch Loss: 4.998356416763272e-06
Epoch 4519, Loss: 0.00018712723976932466, Final Batch Loss: 9.347173909191042e-05
Epoch 4520, Loss: 0.0012186202307020721, Final Batch Loss: 4.9333407332596835e-06
Epoch 4521, Loss: 0.00133447464031633, Final Batch Loss: 0.00017281399050261825
Epoch 4522, Loss: 7.663568430871237e-05, Final Batch Loss: 4.90682759846095e-05
Epoch 4523, Loss: 3.806185668508988e-05, Final Batch Loss: 1.3047585525782779e-05
Epoch 4524, Loss: 0.00044647077265835833, Final Batch Loss: 0.00043957424350082874
Epoch 4525, Loss: 3.

Epoch 4626, Loss: 0.0010162086691707373, Final Batch Loss: 0.000999553594738245
Epoch 4627, Loss: 0.0001393161146552302, Final Batch Loss: 2.5205634301528335e-05
Epoch 4628, Loss: 1.586081452842336e-05, Final Batch Loss: 6.2722101574763656e-06
Epoch 4629, Loss: 1.790584428817965e-05, Final Batch Loss: 8.275054824480321e-06
Epoch 4630, Loss: 0.0006177052255225135, Final Batch Loss: 6.761982149328105e-06
Epoch 4631, Loss: 0.0002481902283761883, Final Batch Loss: 0.0002280553599121049
Epoch 4632, Loss: 4.262460652171285e-05, Final Batch Loss: 3.7215118936728686e-05
Epoch 4633, Loss: 7.11096126906341e-05, Final Batch Loss: 2.9406126486719586e-05
Epoch 4634, Loss: 0.00025723626094986685, Final Batch Loss: 0.00020007217244710773
Epoch 4635, Loss: 0.00023588723138345813, Final Batch Loss: 1.936192120410851e-06
Epoch 4636, Loss: 5.5825334129622206e-05, Final Batch Loss: 1.0439151083119214e-05
Epoch 4637, Loss: 5.121950925968122e-05, Final Batch Loss: 2.277834209962748e-05
Epoch 4638, Loss: 0.0

Epoch 4728, Loss: 1.3912713711761171e-05, Final Batch Loss: 2.242929895146517e-06
Epoch 4729, Loss: 5.224897904554382e-05, Final Batch Loss: 3.58839170075953e-05
Epoch 4730, Loss: 3.951576218241826e-05, Final Batch Loss: 1.5555942809442058e-05
Epoch 4731, Loss: 0.0005027928364143008, Final Batch Loss: 2.3518174202763475e-05
Epoch 4732, Loss: 0.005701682117432938, Final Batch Loss: 2.0999048501835205e-05
Epoch 4733, Loss: 0.00047650201167925843, Final Batch Loss: 0.0004705707833636552
Epoch 4734, Loss: 5.327230633156432e-05, Final Batch Loss: 1.6156047877302626e-06
Epoch 4735, Loss: 0.0015703699257301196, Final Batch Loss: 3.3641638310655253e-06
Epoch 4736, Loss: 2.9457638675012277e-05, Final Batch Loss: 3.4890965707745636e-06
Epoch 4737, Loss: 8.098452553895186e-06, Final Batch Loss: 2.5819833808782278e-06
Epoch 4738, Loss: 4.0491688196198083e-05, Final Batch Loss: 1.6703739674994722e-05
Epoch 4739, Loss: 0.00016481966758874478, Final Batch Loss: 7.223764441732783e-06
Epoch 4740, Loss:

Epoch 4838, Loss: 0.001229279918334214, Final Batch Loss: 1.7966620362130925e-05
Epoch 4839, Loss: 4.612409793480765e-05, Final Batch Loss: 1.6281159332720563e-05
Epoch 4840, Loss: 0.0001688048866981262, Final Batch Loss: 2.7127459816256305e-06
Epoch 4841, Loss: 0.0009775053295015823, Final Batch Loss: 4.08830055675935e-05
Epoch 4842, Loss: 5.433895239548292e-05, Final Batch Loss: 4.151938992436044e-05
Epoch 4843, Loss: 3.517381537676556e-05, Final Batch Loss: 1.0230923180643003e-05
Epoch 4844, Loss: 4.940663029628922e-05, Final Batch Loss: 3.511253908072831e-06
Epoch 4845, Loss: 7.223311058623949e-05, Final Batch Loss: 6.381468119798228e-05
Epoch 4846, Loss: 4.050437200930901e-05, Final Batch Loss: 1.723978311929386e-05
Epoch 4847, Loss: 8.088826916718972e-06, Final Batch Loss: 2.284487209180952e-06
Epoch 4848, Loss: 0.0005407727439887822, Final Batch Loss: 1.4687771908938885e-05
Epoch 4849, Loss: 7.050261365293409e-05, Final Batch Loss: 6.617753388127312e-05
Epoch 4850, Loss: 9.16036

Epoch 4949, Loss: 5.4614312830381095e-05, Final Batch Loss: 2.8911599656566978e-05
Epoch 4950, Loss: 7.066733815008774e-05, Final Batch Loss: 2.8026886866427958e-05
Epoch 4951, Loss: 5.909260471526068e-05, Final Batch Loss: 2.6516770958551206e-05
Epoch 4952, Loss: 9.869483437796589e-05, Final Batch Loss: 6.436775947804563e-06
Epoch 4953, Loss: 1.241203790414147e-05, Final Batch Loss: 6.087212113925489e-06
Epoch 4954, Loss: 2.1496817225852283e-05, Final Batch Loss: 5.488699116540374e-06
Epoch 4955, Loss: 0.0005947577592451125, Final Batch Loss: 0.0004402088525239378
Epoch 4956, Loss: 4.5193228288553655e-05, Final Batch Loss: 9.464085451327264e-06
Epoch 4957, Loss: 0.002623784950174013, Final Batch Loss: 0.002621275605633855
Epoch 4958, Loss: 5.449421769299079e-05, Final Batch Loss: 7.959823051351123e-06
Epoch 4959, Loss: 1.850173248385545e-05, Final Batch Loss: 7.759823347441852e-06
Epoch 4960, Loss: 0.00015461310249520466, Final Batch Loss: 0.00012310284364502877
Epoch 4961, Loss: 1.75

In [10]:
softmax = nn.Softmax(dim = 1)
model.eval()
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 5))

[[39  0  0]
 [ 0 23  0]
 [ 0  0 27]]
              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000        39
           1    1.00000   1.00000   1.00000        23
           2    1.00000   1.00000   1.00000        27

    accuracy                        1.00000        89
   macro avg    1.00000   1.00000   1.00000        89
weighted avg    1.00000   1.00000   1.00000        89



In [11]:
gen = Generator(z_dim = 106)
load_model(gen, "cGAN_UCI_Group_2_gen.param")
gen.eval()

Generator(
  (gen): Sequential(
    (0): Sequential(
      (0): Linear(in_features=106, out_features=80, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): BatchNorm1d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): Linear(in_features=80, out_features=60, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): BatchNorm1d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
    (2): Sequential(
      (0): Linear(in_features=60, out_features=50, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
    (3): Linear(in_features=50, out_features=46, bias=True)
    (4): Tanh()
  )
)

In [12]:
size = len(X_test)
latent_vectors = get_noise(size, 100)
act_vectors = get_act_matrix(size, 3)
usr_vectors = get_usr_matrix(size, 3)

to_gen = torch.cat((latent_vectors, act_vectors[1], usr_vectors[1]), 1)
fake_features = gen(to_gen).detach()

In [13]:
_, preds = torch.max(softmax(model(fake_features.float())), dim = 1)
print(metrics.confusion_matrix(act_vectors[0], preds.cpu()))
print(metrics.classification_report(act_vectors[0], preds.cpu(), digits = 5))

[[26  0  0]
 [ 0 34  0]
 [ 0  0 29]]
              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000        26
           1    1.00000   1.00000   1.00000        34
           2    1.00000   1.00000   1.00000        29

    accuracy                        1.00000        89
   macro avg    1.00000   1.00000   1.00000        89
weighted avg    1.00000   1.00000   1.00000        89



# User Classifier

In [14]:
activities = [1, 3, 4]
users = [7, 8, 11]

X, y = start_data(activities, users, "Subject", sub_features, act_features)

In [15]:
for k in range(len(y)):
    if y[k] == 7:
        y[k] = 0
    elif y[k] == 8:
        y[k] = 1
    else:
        y[k] = 2

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)

model_subject = Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_subject.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [17]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model_subject(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 2.2510942220687866, Final Batch Loss: 1.0841840505599976
Epoch 2, Loss: 2.2959787845611572, Final Batch Loss: 1.1698410511016846
Epoch 3, Loss: 2.281762957572937, Final Batch Loss: 1.1564549207687378
Epoch 4, Loss: 2.2604804039001465, Final Batch Loss: 1.1193715333938599
Epoch 5, Loss: 2.260292172431946, Final Batch Loss: 1.132501482963562
Epoch 6, Loss: 2.2591344118118286, Final Batch Loss: 1.1318106651306152
Epoch 7, Loss: 2.2493975162506104, Final Batch Loss: 1.1259241104125977
Epoch 8, Loss: 2.2487409114837646, Final Batch Loss: 1.1266247034072876
Epoch 9, Loss: 2.243160367012024, Final Batch Loss: 1.1294907331466675
Epoch 10, Loss: 2.2435139417648315, Final Batch Loss: 1.1306999921798706
Epoch 11, Loss: 2.2192611694335938, Final Batch Loss: 1.0988774299621582
Epoch 12, Loss: 2.2338547706604004, Final Batch Loss: 1.1275932788848877
Epoch 13, Loss: 2.203971028327942, Final Batch Loss: 1.0928239822387695
Epoch 14, Loss: 2.2088714838027954, Final Batch Loss: 1.104533910

Epoch 127, Loss: 1.1624908447265625, Final Batch Loss: 0.5936018824577332
Epoch 128, Loss: 1.1042536497116089, Final Batch Loss: 0.5259116291999817
Epoch 129, Loss: 1.1246260404586792, Final Batch Loss: 0.5408570170402527
Epoch 130, Loss: 1.1036645770072937, Final Batch Loss: 0.5110021233558655
Epoch 131, Loss: 1.1107389330863953, Final Batch Loss: 0.5481696128845215
Epoch 132, Loss: 1.1442975401878357, Final Batch Loss: 0.536227822303772
Epoch 133, Loss: 1.1308058500289917, Final Batch Loss: 0.5440545082092285
Epoch 134, Loss: 1.1311856508255005, Final Batch Loss: 0.562324047088623
Epoch 135, Loss: 1.1368414163589478, Final Batch Loss: 0.5903642773628235
Epoch 136, Loss: 1.1383953094482422, Final Batch Loss: 0.5716593265533447
Epoch 137, Loss: 1.0764418244361877, Final Batch Loss: 0.5028005838394165
Epoch 138, Loss: 1.1519526243209839, Final Batch Loss: 0.6086592674255371
Epoch 139, Loss: 1.1045037508010864, Final Batch Loss: 0.5481641888618469
Epoch 140, Loss: 1.2076791524887085, Fin

Epoch 250, Loss: 0.8679121732711792, Final Batch Loss: 0.43086349964141846
Epoch 251, Loss: 0.8672917187213898, Final Batch Loss: 0.4553251564502716
Epoch 252, Loss: 0.90122190117836, Final Batch Loss: 0.47165951132774353
Epoch 253, Loss: 0.8512239754199982, Final Batch Loss: 0.4356773793697357
Epoch 254, Loss: 0.9199870526790619, Final Batch Loss: 0.4986361861228943
Epoch 255, Loss: 0.8515042960643768, Final Batch Loss: 0.46224185824394226
Epoch 256, Loss: 0.8343904316425323, Final Batch Loss: 0.417363703250885
Epoch 257, Loss: 0.8723017275333405, Final Batch Loss: 0.46541017293930054
Epoch 258, Loss: 0.9159546196460724, Final Batch Loss: 0.4733591079711914
Epoch 259, Loss: 0.7927674055099487, Final Batch Loss: 0.365781307220459
Epoch 260, Loss: 0.8197609782218933, Final Batch Loss: 0.3965409994125366
Epoch 261, Loss: 0.795908123254776, Final Batch Loss: 0.37235715985298157
Epoch 262, Loss: 0.8320090174674988, Final Batch Loss: 0.4373854100704193
Epoch 263, Loss: 0.8276866376399994, F

Epoch 378, Loss: 0.3643789291381836, Final Batch Loss: 0.16385409235954285
Epoch 379, Loss: 0.4345874786376953, Final Batch Loss: 0.21674621105194092
Epoch 380, Loss: 0.3762754946947098, Final Batch Loss: 0.1863587200641632
Epoch 381, Loss: 0.37431128323078156, Final Batch Loss: 0.18674233555793762
Epoch 382, Loss: 0.3683106005191803, Final Batch Loss: 0.18573066592216492
Epoch 383, Loss: 0.4150029718875885, Final Batch Loss: 0.20814916491508484
Epoch 384, Loss: 0.3118809461593628, Final Batch Loss: 0.14526918530464172
Epoch 385, Loss: 0.43561387062072754, Final Batch Loss: 0.2780410349369049
Epoch 386, Loss: 0.39130300283432007, Final Batch Loss: 0.19379687309265137
Epoch 387, Loss: 0.38751381635665894, Final Batch Loss: 0.19259124994277954
Epoch 388, Loss: 0.30822083353996277, Final Batch Loss: 0.13041174411773682
Epoch 389, Loss: 0.3985576927661896, Final Batch Loss: 0.23561204969882965
Epoch 390, Loss: 0.4248001277446747, Final Batch Loss: 0.22085458040237427
Epoch 391, Loss: 0.307

Epoch 489, Loss: 0.28437216579914093, Final Batch Loss: 0.13627001643180847
Epoch 490, Loss: 0.28155621886253357, Final Batch Loss: 0.13504379987716675
Epoch 491, Loss: 0.3152358680963516, Final Batch Loss: 0.1462816596031189
Epoch 492, Loss: 0.3253253400325775, Final Batch Loss: 0.16905079782009125
Epoch 493, Loss: 0.31409014761447906, Final Batch Loss: 0.17754115164279938
Epoch 494, Loss: 0.3547675609588623, Final Batch Loss: 0.19936524331569672
Epoch 495, Loss: 0.2314099296927452, Final Batch Loss: 0.10375276952981949
Epoch 496, Loss: 0.30174168199300766, Final Batch Loss: 0.18022973835468292
Epoch 497, Loss: 0.28590138256549835, Final Batch Loss: 0.1298711895942688
Epoch 498, Loss: 0.3097292333841324, Final Batch Loss: 0.19002652168273926
Epoch 499, Loss: 0.24101264774799347, Final Batch Loss: 0.08157339692115784
Epoch 500, Loss: 0.3108999505639076, Final Batch Loss: 0.18640395998954773
Epoch 501, Loss: 0.28922851383686066, Final Batch Loss: 0.1417754739522934
Epoch 502, Loss: 0.28

...

Epoch 4475, Loss: 0.03300483338534832, Final Batch Loss: 0.016572488471865654
Epoch 4476, Loss: 0.029120516031980515, Final Batch Loss: 0.004222739487886429
Epoch 4477, Loss: 0.016685894690454006, Final Batch Loss: 0.011600488796830177
Epoch 4478, Loss: 0.005172166274860501, Final Batch Loss: 0.002522819908335805
Epoch 4479, Loss: 0.016148301190696657, Final Batch Loss: 0.0006937443977221847
Epoch 4480, Loss: 0.014028037898242474, Final Batch Loss: 0.0016846591606736183
Epoch 4481, Loss: 0.026800096267834306, Final Batch Loss: 0.002716858172789216
Epoch 4482, Loss: 0.007310642162337899, Final Batch Loss: 0.0033633087296038866
Epoch 4483, Loss: 0.019943254068493843, Final Batch Loss: 0.006975429132580757
Epoch 4484, Loss: 0.04338900186121464, Final Batch Loss: 0.024808280169963837
Epoch 4485, Loss: 0.01813744381070137, Final Batch Loss: 0.00667875912040472
Epoch 4486, Loss: 0.018946449854411185, Final Batch Loss: 0.0018784810090437531
Epoch 4487, Loss: 0.10010586935095489, Final Batch L

Epoch 4580, Loss: 0.021337989252060652, Final Batch Loss: 0.002394552808254957
Epoch 4581, Loss: 0.03467795019969344, Final Batch Loss: 0.004335572477430105
Epoch 4582, Loss: 0.031234925147145987, Final Batch Loss: 0.0049036904238164425
Epoch 4583, Loss: 0.0454500550404191, Final Batch Loss: 0.0393693782389164
Epoch 4584, Loss: 0.06421983800828457, Final Batch Loss: 0.05720819532871246
Epoch 4585, Loss: 0.035258576506748796, Final Batch Loss: 0.0020017775241285563
Epoch 4586, Loss: 0.011124405544251204, Final Batch Loss: 0.005336157511919737
Epoch 4587, Loss: 0.01463820319622755, Final Batch Loss: 0.004653365351259708
Epoch 4588, Loss: 0.0356032052077353, Final Batch Loss: 0.028658177703619003
Epoch 4589, Loss: 0.015005994588136673, Final Batch Loss: 0.012056841515004635
Epoch 4590, Loss: 0.011724160984158516, Final Batch Loss: 0.0026096096262335777
Epoch 4591, Loss: 0.011243519373238087, Final Batch Loss: 0.007619387935847044
Epoch 4592, Loss: 0.06078202649950981, Final Batch Loss: 0.

Epoch 4696, Loss: 0.04121409170329571, Final Batch Loss: 0.01642524264752865
Epoch 4697, Loss: 0.007205870701000094, Final Batch Loss: 0.00326916784979403
Epoch 4698, Loss: 0.03555078664794564, Final Batch Loss: 0.03351261094212532
Epoch 4699, Loss: 0.005771578289568424, Final Batch Loss: 0.002125235740095377
Epoch 4700, Loss: 0.014413512777537107, Final Batch Loss: 0.005226373206824064
Epoch 4701, Loss: 0.01929409068543464, Final Batch Loss: 0.017790792509913445
Epoch 4702, Loss: 0.01286145020276308, Final Batch Loss: 0.008146189153194427
Epoch 4703, Loss: 0.1686791144311428, Final Batch Loss: 0.13276085257530212
Epoch 4704, Loss: 0.004346345318481326, Final Batch Loss: 0.0023494393099099398
Epoch 4705, Loss: 0.0035076833446510136, Final Batch Loss: 0.0008350068819709122
Epoch 4706, Loss: 0.005381783354096115, Final Batch Loss: 0.0018714995821937919
Epoch 4707, Loss: 0.016653274651616812, Final Batch Loss: 0.004411100875586271
Epoch 4708, Loss: 0.0277512576431036, Final Batch Loss: 0.

Epoch 4801, Loss: 0.03285809699445963, Final Batch Loss: 0.012259642593562603
Epoch 4802, Loss: 0.026746324263513088, Final Batch Loss: 0.009655431844294071
Epoch 4803, Loss: 0.02275104634463787, Final Batch Loss: 0.011553510092198849
Epoch 4804, Loss: 0.02165837027132511, Final Batch Loss: 0.005447475239634514
Epoch 4805, Loss: 0.07520513236522675, Final Batch Loss: 0.008076861500740051
Epoch 4806, Loss: 0.05765140801668167, Final Batch Loss: 0.03992074728012085
Epoch 4807, Loss: 0.04854793194681406, Final Batch Loss: 0.03515706583857536
Epoch 4808, Loss: 0.026325120590627193, Final Batch Loss: 0.01212012767791748
Epoch 4809, Loss: 0.04567462485283613, Final Batch Loss: 0.03640466928482056
Epoch 4810, Loss: 0.029747571796178818, Final Batch Loss: 0.010508708655834198
Epoch 4811, Loss: 0.04854295775294304, Final Batch Loss: 0.010870859026908875
Epoch 4812, Loss: 0.01838819310069084, Final Batch Loss: 0.009718786925077438
Epoch 4813, Loss: 0.027500759344547987, Final Batch Loss: 0.00355

Epoch 4910, Loss: 0.0198309610132128, Final Batch Loss: 0.0027904140297323465
Epoch 4911, Loss: 0.013609740766696632, Final Batch Loss: 0.001139553147368133
Epoch 4912, Loss: 0.02482595667243004, Final Batch Loss: 0.007349472492933273
Epoch 4913, Loss: 0.08970328979194164, Final Batch Loss: 0.07378701120615005
Epoch 4914, Loss: 0.14538953080773354, Final Batch Loss: 0.08846072107553482
Epoch 4915, Loss: 0.008954373188316822, Final Batch Loss: 0.0024651847779750824
Epoch 4916, Loss: 0.021286738105118275, Final Batch Loss: 0.010812920518219471
Epoch 4917, Loss: 0.010158807970583439, Final Batch Loss: 0.005397922825068235
Epoch 4918, Loss: 0.011553057003766298, Final Batch Loss: 0.0056616924703121185
Epoch 4919, Loss: 0.00980545743368566, Final Batch Loss: 0.0023339081089943647
Epoch 4920, Loss: 0.07128095394000411, Final Batch Loss: 0.06656983494758606
Epoch 4921, Loss: 0.017098412849009037, Final Batch Loss: 0.005478803999722004
Epoch 4922, Loss: 0.0129766627214849, Final Batch Loss: 0.

In [18]:
softmax = nn.Softmax(dim = 1)
model_subject.eval()
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model_subject(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 5))

[[33  0  0]
 [ 2 22  1]
 [ 0  0 31]]
              precision    recall  f1-score   support

           0    0.94286   1.00000   0.97059        33
           1    1.00000   0.88000   0.93617        25
           2    0.96875   1.00000   0.98413        31

    accuracy                        0.96629        89
   macro avg    0.97054   0.96000   0.96363        89
weighted avg    0.96793   0.96629   0.96564        89



In [19]:
_, preds = torch.max(softmax(model_subject(fake_features.float())), dim = 1)
print(metrics.confusion_matrix(usr_vectors[0], preds.cpu()))
print(metrics.classification_report(usr_vectors[0], preds.cpu(), digits = 5))

[[38  0  0]
 [ 0 21  5]
 [ 0  0 25]]
              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000        38
           1    1.00000   0.80769   0.89362        26
           2    0.83333   1.00000   0.90909        25

    accuracy                        0.94382        89
   macro avg    0.94444   0.93590   0.93424        89
weighted avg    0.95318   0.94382   0.94339        89

